# Tutorial script for using the ensemble and selectivity maps

In [13]:
from hotspots.hs_ensembles import EnsembleResult, SelectivityResult
from hotspots.hs_io import HotspotReader, HotspotWriter
from hotspots.grid_extension import _GridEnsemble
from pathlib import Path

### What do the ensemble maps settings mean?

1. How ensemble maps are put together:

![Flowchart for compiling the ensemble maps](./notebook_images/ensemble_maps_method.png)

2. Verbose description of the parameters:


 - Frequency: ((number of times score observed at point)/ (number of maps in ensemble))*100
   - For the polar maps, using a frequency threshold is used to remove artefacts of the alignment and "noisy" hotspots that are unlikely to contribute to the binding. For the apolar maps, the default frequency threshold is 0 (meaning to take into account all points), as these effects affect them less due to their larger volume and lack of orientation dependence.
        
 - Combine mode: 
   - To calculate the ensemble maps, the algorithm takes the median (or mean, or max)of the nonzero values for each point in the polar maps that passes the frequency threshold. For the apolar maps, the default is to take the median (or other combine_mode) of all values, including zeros.
        
   - The combine_mode parameter controls how the information at each point is combined. Currently, the default is to take the median (with some modifications for the polar maps), but the 'mean' and 'max' modes could also be useful for exploratory work. 
            
            
* Note: Ensemble maps have not been tested with the positive and negative probes, although the option has been included here to allow exploration and testing.

In [4]:
# Set up parameters for the Ensemble Maps. Shown below are the default values, but the ensemble_settings allow for tweaking them.

ensemble_settings= EnsembleResult.Settings()
# Take the median value across the ensemble at each point in space
ensemble_settings.combine_mode = 'median'

# Include all points in the apolar maps
ensemble_settings.apolar_frequency_threshold = 0.0

# For the polar maps, use only points with nonzero scores in over 20% of the ensemble.
ensemble_settings.polar_frequency_threshold = 20.0

### * Note: The protein models used to calculate the input hotspot results for both the ensemble and selectivity maps need to have been aligned at the binding site of interest prior to the hotspots calculation. This can be done through the CCDC API, as described [here]( https://downloads.ccdc.cam.ac.uk/documentation/API/descriptive_docs/protein.html#superposing-protein-chains-and-binding-sites)


Below is an example for calculating ensemble maps from a set of pre-calculated hotspot results for p38alpha kinase. 

In [9]:
on_target = "p38alpha"

# Load the pre-aligned, pre-calculated results:
target_paths = list(Path(f"{on_target}").glob('*/fullsize_hotspots_3000/out.zip'))

# Create the EnsembleResult object
ensemble = EnsembleResult(hs_results_list=[HotspotReader(p).read() for p in target_paths],
                                    ensemble_id=on_target,
                                    settings=ensemble_settings)

# Calculate the ensemble maps. This can be memory-intensive with large or many grids.
ensemble.make_ensemble_maps(save_grid_ensembles=True)

# Save the ensemble maps as a hotspots.Results object
with HotspotWriter(f"ensemble_{on_target}", grid_extension=".ccp4", zip_results=False) as w:
    w.write(ensemble.ensemble_hotspot_result)


Making the common grids
Stared making arrays
GridEnsemble complete
donor (223, 201, 225)
Making the common grids
Stared making arrays
GridEnsemble complete
acceptor (223, 201, 225)
Making the common grids
Stared making arrays
GridEnsemble complete
apolar (223, 201, 225)
10
14
17


The EnsembleResult object compiles ensemble maps from a list of hotspot results. If the input grids are not the same size, they will be converted to the same size and coordinates via the hotspots.grid_extension.Grid.common_grid() method. The EnsembleResult can keep track of which grid belongs to which hotspot result. It identifies the hotspot results via the identifier of the protein in the input hotspot result (hs_result.protein.identifier). Usually, this is set automatically when reading in the PDB file, but can also be modified manually.

Grids for each probe type are converted into numpy arrays and stacked into a hotspots.grid_extension.\_GridEnsemble object, which is a wrapper around the 4D numpy array that holds the ensemble information for that probe type. Saving the grid ensembles generated in EnsembleResult.make_ensemble_maps() takes up extra memory, but allows for more detailed analysis.

For example, let's see which structures in the ensemble contribute to one of the hotspot features in the acceptor ensemble map:

In [33]:
# Find hotspot features in the acceptor maps:
p38_ensemble_acceptor_grid = ensemble.ensemble_hotspot_result.super_grids['acceptor']
p38_acceptor_gridensemble = ensemble.grid_ensembles['acceptor']

# Convert into numpy array:
p38_ensemble_acceptor_array = _GridEnsemble.array_from_grid(p38_ensemble_acceptor_grid)

# Find the hotspot features using HDBSCAN
p38_ensemble_acceptor_clusters = _GridEnsemble.HDBSCAN_cluster(p38_ensemble_acceptor_array,
                                                              min_cluster_size=7)

# Let's see which structures contribute to cluster 24:
contribs = p38_acceptor_gridensemble.get_contributing_maps(p38_ensemble_acceptor_clusters)[24]

for c in contribs:
    print(f"Structure: {ensemble.index_dict[c[0]]}, points in cluster: {c[1]}")

Structure: 1w7h.A, points in cluster: 11
Structure: 1bl6.A, points in cluster: 10
Structure: 2y8o.A, points in cluster: 23
Structure: 1w84.A, points in cluster: 18
Structure: 1wbo.A, points in cluster: 13


## Using the selectivity maps:

Similar to the ensemble maps, selectivity maps are calculated by a SelectivityResult object, which takes hotspot results as input. Currently, it can only compare two results at a time, but functionality to compare more results is in the works.

Like the ensemble maps, parameters for the selectivity maps can be set via a SelectivityResult.Settings() instance.

Selectivity map parameters:

![Flowchart for compiling the selectivity maps](./notebook_images/selectivity_maps_method.png)

Feature detection in the selectivity maps is performed using the HDBSCAN algorithm

- minimal_cluster_score (float): the minimal score needed for a cluster to be considered selective

            
- cluster_distance_cutoff (float): How far away a selective cluster can be from another selective cluster in the off-target  map (in angstroms). Some dependence on flexibility of binding site. Usually between 1.5 and 3.0

            
- apolar_percentile_threshold (float): If specified, only takes the top percentile of points specified in the difference map. Useful for clustering in the apolar maps, as they tend to be densely populated with low-scoring values

            
- polar_percentile_threshold (float): If specified, only takes the top percentile of points specified in the difference map. Currently not used (i.e. defaults to 0).

            
- minimum_points_cluster_polar(int): Currently, HDBSCAN used for feature detection in the difference maps. This parameter corresponds to the "min_cluster_size" kwarg. Default value is 7 based on retrospective examples, but 5-10 range could be reasonable.

                                                
- minimum_points_cluster_apolar (int): The apolar maps tend to have larger clusters, so the minimum HDBSCAN cluster size is correspondingly larger (default: 27).


Example: calculating selectivity maps for p38a over a related MAPK kinase, ERK2

In [45]:
off_target = "ERK2"

# To save some time, let's load a pre-calculated ensemble.
off_target_ensemble_hotspot_result = HotspotReader(Path(f"ensemble_{off_target}/out")).read()

selectivity_settings = SelectivityResult.Settings()

# Kinases are pretty dynamic, so let's set the cluster_distance_cutoff a bit higher:
selectivity_settings.cluster_distance_cutoff = 3.0

p38alpha_over_ERK2 = SelectivityResult(target_result=ensemble.ensemble_hotspot_result,
                                       other_result=off_target_ensemble_hotspot_result)
p38alpha_over_ERK2.make_selectivity_maps()

with HotspotWriter(f"selectivity_{on_target}_over_{off_target}",grid_extension=".ccp4", zip_results=False) as wr:
    wr.write(p38alpha_over_ERK2.selectivity_result)

FileNotFoundError: [Errno 2] No such file or directory: 'ensemble_ERK2/out'